# Subqueries
## Principe des Subqueries
- Une subquery est une requête imbriquée à l'intérieur d'une autre requête
- Elles permettent de créer des conditions complexes, de filtrer des résultats ou de réaliser des calculs basés sur d'autres requêtes
- Les résultats de la subquery sont traités comme une table normale par la requête principale
- Attention aux performances, il y a un tradeoff entre optimisation et lisibilité malheureusement
- L'exercice 4 constitue un exemple
  
## Syntaxe

- La subquery peut être utilisée dans différentes parties de la requête principale : SELECT, FROM, WHERE

- Syntaxe avec WHERE : 
    ```sql
    SELECT colonne
    FROM table
    WHERE colonne [condition] (SELECT colonne FROM table WHERE condition) ;
    ```

## Subquery dans une clause WHERE

- Sélectionner tous les produits dont le prix unitaire est supérieur à la moyenne :
```sql
SELECT product_name, unit_price
FROM products
WHERE unit_price > (SELECT AVG(unit_price) FROM products) ;
```

### Utilisation de ANY
- Permet de comparer une valeur avec un ensemble de valeurs retourné par une subquery

- Trouver les produits dont le unit_price est supérieur à celui de n'importe quel produit de la catégorie 2 :

```sql
SELECT product_name, unit_price
FROM products
WHERE unit_price > ANY (SELECT unit_price FROM products WHERE category_id = 2) ;
```

### Utilisation de ALL
- Compare une valeur avec toutes les valeurs retournées par une subquery.
- Sélectionner les produits dont le prix est supérieur à tous les prix des produits de la catégorie 3 :
```sql
SELECT product_name, unit_price
FROM products
WHERE unit_price > ALL (SELECT unit_price FROM products WHERE category_id = 3) ;
```
### Utilisation de EXISTS
- Vérifie l'existence de lignes retournées par une subquery.
- Trouver les produits qui ont au moins une commande en attente (units_on_order > 0) :
```sql
SELECT product_name
FROM products
WHERE EXISTS (SELECT 1 FROM products WHERE units_on_order > 0 AND products.product_id = product_id) ;
```

## Subquery dans une clause SELECT 

- Exemple : sélectionner des données et ajouter sur chaque ligne des résultats une valeur concernant un groupe : 

- Ici on sélectionne des infos produits, et on rajoute dans une colonne le nombre total de produits dans la catégorie du produit de la ligne
  
```sql
SELECT 
    p.product_name, 
    p.unit_price, 
    p.category_id,
    (SELECT COUNT(*)
     FROM products sub_p
     WHERE sub_p.category_id = p.category_id) AS total_products_in_category
FROM products p;
```

# Utilisation d'une subquery dans la Clause FROM

- Exemple : sélectionner des données et ajouter sur chaque ligne des résultats une valeur concernant un groupe

- Ici, on sélectionne les produits et leur prix, s'ils font partie d'une catégorie "chère", c'est à dire dont le prix moyen est supérieur à 50

```sql
SELECT 
    p.product_name, 
    p.unit_price
FROM products p, 
     (SELECT category_id
      FROM products
      GROUP BY category_id
      HAVING AVG(unit_price) > 50) AS expensive_categories
WHERE p.category_id = expensive_categories.category_id;

```

#### Work in progress - tentative to simplify stuff
```sql
SELECT product_name
FROM (SELECT product_name, unit_price, category_id
      FROM products
      GROUP BY category_id
      HAVING AVG(unit_price) > 50) AS expensive_categories;
```